# 使用LangChain调用LLM

## 安装LangChain

使用以下命令安装LangChain。由于使用DeepSeek模型，因此要同时安装`langchain-deepseek`。

```python
%pip install langchain langchain-deepseek
```

> ⚠️注意：
> 如果使用的是虚拟环境(例如Conda或者virtualenv)，应该使用魔法命令`%pip install`，而非shell命令`!pip install`，因为前者会将软件
> 包安装至当前环境，而后者会将软件包安装至默认环境。

使用DeepSeek时，要设置环境变量`DEEPSEEK_API_KEY`。使用其它的大模型时，也要在环境变量中设置好对应的API_KEY，比如使用OpenAI的模型时
要设置`OPENAI_API_KEY`。

## 使用`init_chat_model`来创建LLM

一种方法是调用`init_chat_model`函数来根据传入的参数初始化对应的模型类。然后执行模型的`invoke`方法来进行大模型调用。

In [21]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model(model='deepseek-chat', model_provider='deepseek')

messages = [
    SystemMessage("将输入的文字翻译成英语。"),
    HumanMessage("你好。"),
]

message = model.invoke(messages)
print(message.content)

Hello.


## 使用Stream进行输出

如果要流式的输出模型的返回内容，需要循环调用`stream`方法。

In [22]:
for chunk in model.stream(messages):
    print(chunk.content, end='|')

|Hello|.||

## 直接构造LLM类

另一种方式是直接构造一个模型对象。

In [23]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import HumanMessage, SystemMessage

messages = [SystemMessage('请将输入的文字翻译成英文。'),
            HumanMessage('您好。')]

ds_model = ChatDeepSeek(model='deepseek-chat')
message = ds_model.invoke(messages)
print(message.content)

Hello.


## 使用OpenAI格式输入

除了通过Message列表的形式作为输入，还可以以OpenAI的字典格式来作为输入。

In [3]:
from langchain_deepseek import ChatDeepSeek

messages = [
    {'role': 'system', 'content': '请将输入的文字翻译成英文。'},
    {'role': 'user', 'content': '您好。'},
    ]

ds_model = ChatDeepSeek(model='deepseek-chat')
message = ds_model.invoke(messages)
print(message.content)

Hello.


 ## 使用提示词模板

 实际应用过程中，提示词往往都不是固定的，而是会根据具体的情况变化。langchain提供了`ChatPromptTemplate`来应对这种情况。

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', '请将输入的文字翻译成{language}'),
        ('user', '{text}')
    ]
)

prompt = prompt_template.invoke({'language': '日语', 'text': '您好'})

ds_model = ChatDeepSeek(model='deepseek-chat')
ds_model.invoke(prompt)

AIMessage(content='こんにちは', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 11, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': 'cb2d1824-eb33-455a-a7ec-fbe3ab0ace20', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b0f04f3-dbe0-4dda-8cd3-5768dc276701-0', usage_metadata={'input_tokens': 11, 'output_tokens': 5, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})